In [ ]:
import os
import numpy as np

## 5.19 Rotate a 2D Array 

Image rotation is a fundamental operation in computer graphics. Figure 5.4 illustrates the rotation operation on a 2D array representing a bit-map of an image. Specifically, the image is rotated by 90 degrees clockwise.

Write a function that takes as input an n x n 2D array, and rotates the array by 90 degrees clockwise.
Hint: Focus on the boundary elements.

In [ ]:
def rotate_90_clock_np(arr):
    arr = np.array(arr)
    arr_rot = np.zeros(arr.shape)
    for old_row_ind, new_col_ind in enumerate(range(len(arr))[::-1]):
        arr_rot[:, new_col_ind] = arr[old_row_ind, :]
    return arr_rot


def rotate_coords_90_clock(i, j, n):
    i_rot = j
    j_rot = n - 1 - i
    return i_rot, j_rot


def rotate_corners(arr, i_start, j_start):
    n = len(arr)
    i_orig, j_orig = i_start, j_start
    reassign_val = arr[i_orig][j_orig]
    for _ in range(4):
        i_rot, j_rot = rotate_coords_90_clock(i_orig, j_orig, n)
        temp = arr[i_rot][j_rot]
        arr[i_rot][j_rot] = reassign_val
        reassign_val = temp
        i_orig, j_orig = i_rot, j_rot

        
def rotate_square(arr, ind_start):
    i_start = ind_start
    for j_start in range(ind_start, len(arr) - ind_start - 1):
        rotate_corners(arr, i_start, j_start)

        
def rotate_90_clock_inplace(arr):
    num_square = len(arr) / 2
    for ind_start in range(num_square):
        rotate_square(arr, ind_start)

        
def rotate_90_clock(arr):
    n = len(arr)
    arr_rot = [[None] * n for i in range(n)]
    for i in range(n):
        for j in range(n):
            i_rot, j_rot = rotate_coords_90_clock(i, j, n)
            arr_rot[i_rot][j_rot] = arr[i][j]
    return arr_rot


def rotate(arr):
    for i in xrange(len(arr)/2):
#         for j in xrange(len(arr)/2 + len(arr)%2):
        for j in xrange(i, len(arr) - i - 1):
            rotate_corners(arr,i,j)
    return arr

In [ ]:
# arr = [[i] * n for i in range(5)]
# arr = [[1, 2], [3, 4]]
# arr = [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16]]
arr = [[1, 2, 3, 4, 140], [5, 6, 7, 8, 130], [9, 10, 11, 12, 120], [13, 14, 15, 16, 110], [17, 18, 19, 20, 100]]

# arr_rot = rotate_90_clock(arr)

# print(np.array(arr))
# print('')
# print(np.array(arr_rot))

print(np.array(arr))
# rotate_90_clock_inplace(arr)
rotate(arr)
print('')
print(np.array(arr))

## 5.13 Sample Online Data 

This problem is motivated by the disgn of a packet sniffer that provides a uniform sample of packets for a network session.

Design a program that takes as input a size k, and reads packets, coninuously maintaining a uniform random subset of size k of the read packets.

Hint: Suppose you have a procedure which selects k packets from the first n >= k packets as specified. How would you deal with the (n+1)st packet?

In [ ]:
import random

class PacketSampler:
    
    def __init__(self, k):
        self.k = k
        self.n = 0
        self.subset = []
        
    def receive_packet(self, p):
        if len(self.subset) < self.k:
            self.subset.append(p)
        else:
            replace_prob = float(self.k) / (self.n + 1)
            if random.random() < replace_prob:
                replace_ind = random.randint(0, self.k - 1)
                self.subset[replace_ind] = p
        self.n += 1


class PacketSamplerWrong:
    
    def __init__(self, k):
        self.k = k
        self.n = 0
        self.subset = []
        
    def receive_packet(self, p):
        if len(self.subset) < self.k:
            self.subset.append(p)
        else:
            replace_prob = random.random()
            if random.random() < replace_prob:
                replace_ind = random.randint(0, self.k - 1)
                self.subset[replace_ind] = p
        self.n += 1


def get_packet_subset(n, packet_sampler):
    for i in range(n):
        packet_sampler.receive_packet(i)
    return packet_sampler.subset


def get_simulated_distribution(n, k, sampler_name, num_trials):
    if sampler_name == 'right':
        Sampler = PacketSampler
    elif sampler_name == 'wrong':
        Sampler = PacketSamplerWrong
    else:
        raise ValueError('Unrecognized sampler name.')
    
    packet_counter = {p: 0 for p in range(n)}
    for trial_ind in range(num_trials):
        subset = get_packet_subset(n, Sampler(k))
        for p in subset:
            packet_counter[p] += 1
    
    return packet_counter

In [ ]:
num_trials = 500000
n = 10
k = 3
sampler_name = 'wrong'

packet_counts = get_simulated_distribution(n=n, k=k, sampler_name=sampler_name, num_trials=num_trials)
num_exp = num_trials / n * k

for count in packet_counts.itervalues():
    pct_dev = 100.0 * abs(num_exp - count) / num_exp
    print(pct_dev)

## 5.18 Compute the Spiral Ordering of a 2D array 

In [ ]:
def change_direction(increment):
    increment_new = increment[::-1]
    if abs(increment[0]) == 1:
        increment_new[1] *= -1
    return increment_new

def spiralize(arr):
    i_bounds = [0, len(arr) - 1]
    j_bounds = [0, len(arr[0]) - 1]
    increment = [0, 1]
    i = 0
    j = 0
    arr_spiral = []

    while i_bounds[1] >= i_bounds[0]:

        arr_spiral.append(arr[i][j])

        i_prop = i + increment[0]
        j_prop = j + increment[1]

        if j_prop > j_bounds[1]:
            i_bounds[0] += 1
        elif j_prop < j_bounds[0]:
            i_bounds[1] -= 1
        elif i_prop > i_bounds[1]:
            j_bounds[1] -= 1
        elif i_prop < i_bounds[0]:
            j_bounds[0] += 1
        else:
            i = i_prop
            j = j_prop
            continue

        increment = change_direction(increment)
        i += increment[0]
        j += increment[1]
        
    return arr_spiral

In [ ]:
arr = [[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 16]]

print(np.array(arr))
print('')
print(spiralize(arr))

## 6.13: Find the first occurrence of a substring 

A good string search algorithm is fundamental to the performance of many applications. Several clever algorithms have been proposed for string search, each with its own trade-offs. As a result, there is no single perfect answer. If someone asks you this question in an interview, the best way to approach this problem would be to work through one good algorithm in detail and discuss at a high level other algorithms.

Given two strings s (the "search string") and t (the "text"), find the first occurrence of s in t.

Hint: Form a signature from a string

In [ ]:
from collections import deque


def strings_are_same(str1, str2):
    for c1, c2 in zip(str1, str2):
        if c1 != c2:
            return False
    return True


class RollingHasher:
    """Following https://en.wikipedia.org/wiki/Rabin%E2%80%93Karp_algorithm"""
    
    def __init__(self, substring_len, base=256, prime_modulus=101):
        self.substring_len = substring_len
        self.base = base
        self.prime_modulus = prime_modulus
        self._char_q = deque()
        self._hash = 0
        
    def _add_to_hash(self, c):
        # O(1) time
        self._hash = (self._hash * self.base + ord(c)) % self.prime_modulus
        self._char_q.append(c)
        
    def _remove_first_in_from_hash(self):
        # O(substring_len) time
        left_base_offset = 1
        for i in range(self.substring_len - 1):
            left_base_offset *= self.base 
            left_base_offset %= self.prime_modulus
        remove_val = ord(self._char_q.popleft())
        self._hash = self._hash + self.prime_modulus - remove_val * left_base_offset  # adding extra mod val ensures result isn't negative
        
    def get_hash(self, next_letter):
        # O(substring_len) time
        if len(self._char_q) == self.substring_len:
            hash_sub = self._remove_first_in_from_hash()
        self._add_to_hash(next_letter)
        return self._hash if len(self._char_q) == self.substring_len else None


def find_substring_start_ind_simple(main_string, substring):
    substring_len = len(substring)
    for start_ind in range(0, len(main_string) - substring_len):
        if strings_are_same(substring, main_string[start_ind:start_ind + substring_len]):
            return start_ind
    return None


def find_substring_start_ind_set(main_string, substring):
    substring_hash = set([substring])
    substring_len = len(substring)
    for start_ind in range(0, len(main_string) - substring_len):
        if main_string[start_ind:start_ind + substring_len] in substring_hash:
            return start_ind
    return None

        
def find_substring_start_ind_rabin_karp(main_string, substring):
    # O(substring_len * string_len)
    sub_hasher = RollingHasher(len(substring))
    substring_hash = [sub_hasher.get_hash(c) for c in substring][-1]
    main_hasher = RollingHasher(len(substring))
    for i, c in enumerate(main_string):
        end_ind = i+1
        start_ind = end_ind - len(substring)
        if substring_hash == main_hasher.get_hash(c) and strings_are_same(substring, main_string[start_ind:end_ind]):
            return start_ind
    return None

In [ ]:
main_string = 'gshgiuregbisnreipngseproingseonvdk;nfvdklldksfl'
substring = 'gbisn'

print(find_substring_start_ind_simple(main_string, substring))
print(find_substring_start_ind_set(main_string, substring))
print(find_substring_start_ind_rabin_karp(main_string, substring))

## 6.6: Reverse all the words in a sentence

Given a string containing a set of words separated by whitespace, we would like to transform it to a string in which the words appear in reverse order. For example, "Alice likes Bob" transforms to "Bob likes Alice". We do not need to keep the original string.

Implement a function for reversing the words in a string s.

Hint: It's difficult to solve this with one pass.

In [ ]:
def reverse_words_simple(sentence):
    words = []
    word = ''
    for c in sentence:
        if c == ' ':
            words.append(word)
            words.append(' ')
            word = ''
        else:
            word += c
    words.append(word)
        
    sentence_rev = ''
    while len(words) > 0:
        sentence_rev += words.pop()
        
    return sentence_rev

def reverse_words_prealloc(sentence):
    end_ind = len(sentence)
    word = ''
    sentence_rev = [' '] * len(sentence)
    for c in sentence:
        if c == ' ':
            start_ind = end_ind-len(word)
            sentence_rev[start_ind:end_ind] = word
            end_ind = start_ind - 1
            word = ''
        else:
            word += c
    sentence_rev[end_ind-len(word):end_ind] = word
        
    return ''.join(sentence_rev)

In [ ]:
sentence = 'Bob likes Alice'
print(reverse_words_simple(sentence))
print(reverse_words_prealloc(sentence))

## 6.7: Compute all mnemonics for a phone number 

Each digit, apart from 0 and 1, in a phone keypad corresponds to one of hte three or four letters of the alphabet, as shown in figure 6.1. Since words are easier to remember than number, it is natural to ask if a 7 or 10-digit phone number can be represented by a word. For examples, "2276696" corresponds to "ACRONYM" as well as "ABPOMZN",

Write a program which takes as input a phone number, specified as a string of digits, and returns all possible character sequences that correspond to the phone number. The cell phone keypad is specified by a mapping that takes a digit and returns the corresponding set of characters. The character sequences do not have to be legal words or phrases.

Hint: Use recursion.

In [ ]:
DIGIT_LETTERS = {
    '2': ['A', 'B', 'C'],
    '3': ['D', 'E', 'F'],
    '4': ['G', 'H', 'I'],
    '5': ['J', 'K', 'L'],
    '6': ['M', 'N', 'O'],
    '7': ['P', 'Q', 'R', 'S'],
    '8': ['T', 'U', 'V'],
    '9': ['W', 'X', 'Y', 'Z']
}


def get_mnemonics(digits):
    if len(digits) == 1:
        return DIGIT_LETTERS[digits]
    
    prev_sequences = get_mnemonics(digits[:-1])
    digit = digits[-1]
    letters = DIGIT_LETTERS[digit]

    new_sequences = []
    for seq in prev_sequences:
        for letter in letters:
            new_sequences.append(seq + letter)
        
    return new_sequences

In [ ]:
digits = '23'
print(get_mnemonics(digits))

## 6.4: Replace and Remove 

Consider the following two rules that are to be applied to an array of characters.
* Replace each 'a' by two 'd's
* Delete each entry containing a 'b'
For example, applying these rules to the array [a,c,d,b,c,a] results in the array [d,d,c,d,c,d,d]

Write a program which takes as input an array of characters, and removes each 'b' and replaces each 'a' by two 'd's. Specifically, along with the array, you are provided an integer-valued size. Size denotes the number of entries of hte array that the operation is to be applied to. You do not have to worry about preserving subsequent entries. For example, if the array is [a,b,a,c,_] and the size is 4, then you can return [d,d,d,d,c]. You can assume there is enough space int he array to hold the final result.

Hint: Consider performing multiple passes on s.

In [ ]:
from collections import deque


def replace_and_remove(arr, size):
    q = deque()

    for i, c in enumerate(arr[:size]):
        if c == 'a':
            q.append('d')
            q.append('d')
        elif c != 'b':
            q.append(c)
        arr[i] = q.popleft()
    
    while len(q) > 0:
        i += 1
        arr[i] = q.popleft()
        
    return arr


def replace_and_remove_inplace(arr, size):
    # Get rid of b's first to make sure there's enough room for the extra d's
    replace_ind = 0
    num_a = 0
    for i, c in enumerate(arr[:size]):
        arr[i] = '_'
        if c != 'b':
            arr[replace_ind] = c
            replace_ind += 1
            if c == 'a':
                num_a += 1
        else:
            size -= 1
    
    # Add in d's
    cur_ind = size + num_a - 1
    for c in arr[:size][::-1]:
        if arr[cur_ind] == 'a':
            arr[cur_ind] = 'd'
            arr[cur_ind - 1] = 'd'
            cur_ind -= 2
        else:
            arr[cur_ind] = c
            cur_ind -= 1

    return arr

In [ ]:
arr = ['a','b','a','c','_']
# arr = ['a','a','a','b','b','b']
size = 4
# print(replace_and_remove(arr, size))
print(replace_and_remove_inplace(arr, size))

## 7.1 Merge Two Sorted Lists 

Consider two singly linked lists in which each node holds a number. Assume the lists are sorted, i.e., number isn the list appear in ascending order within each list. The merge of the two lists is a list consisting of the nodes of the two lists in which numbers appear in ascending order. Merge is illustrated in Figure 7.3.

Write a program that takes two lists, assumed to be sorted, and returns their merge. The only field your program can change in a node is its next field.

Hint: Two sorted arrays can be merged using two indices. For lists, take care when one iterator reaches the end.

In [ ]:
def merge_lists(ll1, ll2):
    if ll2.value < ll1.value:
        ll1, ll2 = ll2, ll1
    ll1_cur = ll1
    ll2_cur = ll2
    
    while ll2_cur is not None:
        # Advance ll1 to just before ll2 becomes smaller
        while ll1_cur.next is not None and ll1_cur.next.value < ll2_cur.value:
            ll1_cur = ll1_cur.next
        
        # Add the rest of ll2 if at the end of ll1
        if ll1_cur.next is None and ll2_cur is not None:
            ll1_cur.next = ll2_cur
            break
        
        # Pop from ll2 and insert into ll1
        ll2_next = ll2_cur.next
        ll2_cur.next = ll1_cur.next
        ll1_cur.next = ll2_cur
        ll2_cur = ll2_next
            
    return ll1

In [ ]:
import sys
sys.path.append('C:\\Users\\David\\Documents\\github_repos\\eopi\\seven')
from linked_list import LinkedList

lili2 = LinkedList([1, 2, 3, 8, 29, 1000])
lili1 = LinkedList([4, 18, 19, 72, 80, 99])
# lili2 = LinkedList([1, 2, 4, 7])
# lili1 = LinkedList([1, 4, 4, 5])

print(merge_lists(lili1, lili2).as_list())

## 7.3 Test for Cyclicity 

Although a linked list is supposed to be a sequence of nodes ending in null, it is possible to create a cycle in a linked list by making the next field of an element reference to one of the earlier nodes. 

Write a program that takes the head of a singly linked list and returns null if there does not exist a cycle, and the node at the start of the cycle, if a cycle is present. (You do not know the length of the list in advance.)

Hint: Consider using two iterators, one fast and one slow.

Fred Note: The first time you solve this problem ignore the hint and use as much extra memory as you want. Then try to solve it in place with O(1) memory and use the hint.

In [ ]:
def find_cycle_naive(ll):
    prev_nodes = set()
    ll_cur = ll
    while ll_cur is not None:
        if ll_cur not in prev_nodes:
            prev_nodes.add(ll_cur)
            ll_cur = ll_cur.next
        else:
            return ll_cur
    return None

def find_loop_node(ll):
    ll_cur_slow = ll
    ll_cur_fast = ll.next
    while None not in [ll_cur_slow, ll_cur_fast]:
        if ll_cur_slow == ll_cur_fast:
            return ll_cur_slow
        ll_cur_slow = ll_cur_slow.next
        ll_cur_fast = ll_cur_fast.next
        ll_cur_fast = None if ll_cur_fast is None else ll_cur_fast.next
    return None

def find_loop_len(loop_node):
    len = 1
    cur_node = loop_node.next
    while cur_node != loop_node:
        len += 1
        cur_node = cur_node.next
    return len

def find_cycle_inplace(ll):
    loop_node = find_loop_node(ll)
    if loop_node is None:
        return None
    
    loop_len = find_loop_len(loop_node)
    print(loop_len)
    
    ll1_cur = ll
    ll2_cur = ll
    for i in range(loop_len):
        ll2_cur = ll2_cur.next
    while ll1_cur != ll2_cur:
        ll1_cur = ll1_cur.next
        ll2_cur = ll2_cur.next
    
    return ll1_cur

In [ ]:
import sys
sys.path.append('C:\\Users\\David\\Documents\\github_repos\\eopi\\seven')
from linked_list import LinkedList

lili1 = LinkedList([1, 41, 4, 15, 5, 7])
lili1.next.next.next.next.next = lili1.next.next

# ll_cycle = find_cycle_naive(lili1)
ll_cycle = find_cycle_inplace(lili1)
print(ll_cycle.value)

## Delete a node from a singly linked list

Given a node in a singly linked list, deleting it in O(1) time appears impossible because its predecessor's next field has to be updated. Surprisingly, it can be done with one small caveat-the node to delete cannot be the last one in the list and it is easy to copy the value part of a node.

Write a program which deletes a node in a singly linked list. The input node is guaranteed not to be the tail node.

Hint: Instead of deleting the node, can you delete its successor and still achieve the desired configuration?

In [ ]:
def delete_node(ll):
    ll.value = ll.next.value
    ll.next = ll.next.next

In [ ]:
import sys
sys.path.append('C:\\Users\\David\\Documents\\github_repos\\eopi\\seven')
from linked_list import LinkedList

lili1 = LinkedList([1, 41, 4, 15, 5, 7])
delete_node(lili1.next.next)
print(lili1)

## 7.8 Remove Duplicates from a Sorted List 

This problem is concerned with removing duplicates from a sorted list of integers. See Figure 7.8 for an example.

Write a program that takes as input a singly linked list of integers in sorted order, and removes duplicates from it. The list should be sorted.

Hint: Focus on the successor fields which have to be updated.

In [ ]:
def remove_duplicates(ll):
    ll_cur = ll
    while ll_cur is not None and ll_cur.next is not None:
        if ll_cur.value == ll_cur.next.value:
            ll_cur.next = ll_cur.next.next
        else:
            ll_cur = ll_cur.next
    return ll

In [ ]:
import sys
sys.path.append('C:\\Users\\David\\Documents\\github_repos\\eopi\\seven')
from linked_list import LinkedList

lili1 = LinkedList([1, 4, 4, 5, 5, 5, 5, 5, 7])
remove_duplicates(lili1)
print(lili1)

## 8.1 Implement a Stack with Max API 

Design a stack that includes a max operation, in addition to push and pop. The max method should return the maximum value stored in the stack.

Hint: Use additional storage to track the maximum value.

In [207]:
class Node:
    
    def __init__(self, val, next):
        self.val = val
        self.next = next

        
class Stack:
    
    def __init__(self):
        self._tail = None
        
    def __str__(self):
        return '->'.join([str(val) for val in self.to_list()])
    
    def to_list(self):
        vals = []
        node = self._tail
        while node is not None:
            vals.append(node.val)
            node = node.next
        return vals[::-1]
        
    def push(self, val):
        self._tail = Node(val, self._tail)
        return self
    
    def pop(self):
        if self._tail is None:
            return None
        tail_val = self._tail.val
        self._tail = self._tail.next    
        return tail_val
    
    def peek(self):
        if self._tail is None:
            return None
        else:
            return self._tail.val
        
        
class MaxStack1:
    """
    time: O(1) push, O(N) pop
    space: O(N) overall, O(1) extra for max
    """
    
    def __init__(self):
        self._tail = None
        self._max = None
        
    def __str__(self):
        vals = []
        node = self._tail
        while node is not None:
            vals.append(str(node.val))
            node = node.next
        vals = vals[::-1]
        return '->'.join(vals)
        
    def push(self, val):
        self._tail = Node(val, self._tail)
        self._max = max(self._max, val)
        return self
    
    def pop(self):
        tail_val = self._tail.val
        self._tail = self._tail.next
        
        if self._tail is None:
            self._max = None
        elif tail_val == self._max:
            self._max = self._tail.val
            node = self._tail.next
            while node is not None:
                self._max = max(node.val, self._max)
                node = node.next
        
        return tail_val
        
    def max(self):
        return self._max
    

class MaxStack2:
    """
    time: O(1) push, O(1) pop
    space: O(N) overall, O(N) extra for max
    """
    
    def __init__(self):
        self._stack = Stack()
        self._max_stack = Stack()
        
    def __str__(self):
        return str(self._stack)
        
    def push(self, val):
        self._stack.push(val)
        if self._stack.peek() >= self._max_stack.peek():
            self._max_stack.push(val)        
        return self
    
    def pop(self):
        tail_val = self._stack.pop()
        if tail_val == self._max_stack.peek():
            self._max_stack.pop()
        return tail_val
        
    def max(self):
        return self._max_stack.peek()
    
    
class MaxStack3:
    """
    time: O(1) push, O(1) pop
    space: O(N) overall, O(1) extra for max
    """
    
    def __init__(self):
        self._tail = None
        self._max = None
        
    def __str__(self):
        return '->'.join([str(val) for val in self.to_list()])
    
    def to_list(self):
        vals = []
        node = self._tail
        while node is not None:
            vals.append(str(node.val))
            node = node.next
        return vals[::-1]
        
    def push(self, val):
        if self._max is None:
            self._max = val
        elif val > self._max:
            temp = 2 * val - self._max  # encode previous max in stored value
            self._max = val
            val = temp
        self._tail = Node(val, self._tail)
        return self
    
    def pop(self):
        tail_val = self._tail.val
        self._tail = self._tail.next
        if tail_val > self._max:
            prev_max = 2 * self._max - tail_val  # decode previous max      
            tail_val = self._max
            self._max = prev_max
        return tail_val
    
    def peek(self):
        if self._tail is None:
            return None
        else:
            return min(self._max, self._tail.val)
        
    def max(self):
        return self._max

In [74]:
# s = MaxStack1()
# s = MaxStack2()
s = MaxStack3()

s.push(5)
print('max', s.max())
s.push(7)
print('max', s.max())
s.push(3)
print('max', s.max())
s.push(7)
print('max', s.max())

print(s)

print('max', s.max())
print('removed', s.pop())
print('max', s.max())
print('removed', s.pop())
print('max', s.max())
print('removed', s.pop())
print('max', s.max())
print('removed', s.pop())

('max', 5)
('max', 7)
('max', 7)
('max', 7)
5->9->3->7
('max', 7)
('removed', 7)
('max', 7)
('removed', 3)
('max', 7)
('removed', 7)
('max', 5)
('removed', 5)


## 8.3 Test a string over "{,},(,),[,]" for Well-Formedness 

A string over the characters "{,},(,),[,]" is said to be well-formed if the different types of brackets match in the correct order.

Write a program that tests if a string made up of these characters if well-formed.

Hint: Which left parenthesis does a right parenthesis match with?

In [145]:
PAREN_MAP = {'}': '{', ')': '(', ']': '['}

def check_parens(s):
    paren_stack = Stack()
    for c in s:
        if c in PAREN_MAP.values():
            paren_stack.push(c)
        elif c in PAREN_MAP.keys():
            opening_paren = paren_stack.pop()
            if PAREN_MAP[c] != opening_paren:
                return False
    return paren_stack.peek() is None

In [147]:
test_str = '{ab[()]c}'
check_parens(test_str)

True

## 8.6 Compute Buildings with a Sunset View 

You are given a series of buildings that have windows facing west. The buildings are in a straight line, and any building which is to the east of a building of equal or greater height cannot view the sunset.

Design an algorithm that processes buildings in east-to-west order and returns the set of buildings which view the sunset. Each building is specified by its height.

Hint: When does a building not have a sunset view?

In [82]:
def get_sunset_view_buildings(heights):
    view_buildings = Stack()
    for height in heights:
        while height >= view_buildings.peek() and view_buildings.peek() is not None:
            view_buildings.pop()
        view_buildings.push(height)
    return view_buildings

east_to_west_heights = [5, 7, 4, 3, 3, 2]
print(get_sunset_view_buildings(east_to_west_heights).to_list())

[7, 4, 3, 2]


## 8.8 Implement a Circular Queue 

A queue can be implemented using an array and two additional fields, the beginning and the end indices. This structure is sometimes referred to as a circular queue. Both enqueue and dequeue have O(1) time complexity. If the array is fixed, there is a maximum number of entries that can be stored. If the array is dynamically resized, the total time for m combined enqueue and dequeue operations is O(m).

Implement a queue API using an array for storing elements. Your API should include a constructor function, which takes as argument the initial capacity of the queue, enqueue and dequeue functions, and a function which returns the number of elements stored. Implement dynamic resizing to support storing an arbitrarily large number of elements.

Hint: Track the head and tail. How can you differentiate a full queue from an empty one?

In [141]:
class CircularQueue1:
    
    def __init__(self, initial_capacity):
        self.initial_capacity = initial_capacity
        self._arr = [None] * initial_capacity
        self._head_ind = 0  # most recent
        self._tail_ind = 0  # least recent
        self._num_elements = 0
    
    def __str__(self):
        return '->'.join([str(val) for val in self.to_list()])
    
    def _advance_ind(self, ind):
        return ind + 1 if ind < len(self._arr) - 1 else 0
            
    def _resize_if_necessary(self):            
        if self._num_elements == len(self._arr):
            self._arr.append(None)
            if self._tail_ind > 0:
                for i in range(0, self._head_ind+1):
                    self._arr[i - 1], self._arr[i] = self._arr[i], None
                self._head_ind -= 1
                if self._head_ind < 0:
                    self._head_ind = len(self._arr) - 1
                
    def to_list(self):
        ind = self._tail_ind
        vals = []
        while True:
            vals.append(self._arr[ind])
            if ind == self._head_ind:
                break
            ind = self._advance_ind(ind)
        return vals[::-1]
        
    def enqueue(self, val):
        self._resize_if_necessary()
        if self._num_elements > 0:
            self._head_ind = self._advance_ind(self._head_ind)
        self._arr[self._head_ind] = val
        self._num_elements += 1
        return self
    
    def dequeue(self):
        val = self._arr[self._tail_ind]
        self._tail_ind = self._advance_ind(self._tail_ind)
        self._num_elements -= 1
        return val
    
    def get_num_elements(self):
        return self._num_elements
    
    
class CircularQueue2:
    
    def __init__(self, initial_capacity):
        self.initial_capacity = initial_capacity
        self._arr = [None] * initial_capacity
        self._head_ind = 0  # most recent
        self._tail_ind = 0  # least recent
        self._num_elements = 0
    
    def __str__(self):
        return '->'.join([str(val) for val in self.to_list()])
    
    def _advance_ind(self, ind):
        return ind + 1 if ind < len(self._arr) - 1 else 0
            
    def _resize_if_necessary(self):            
        if self._num_elements == len(self._arr):
            self._arr.extend([None] * self._num_elements)
            for i in range(self._tail_ind):
                self._arr[self._num_elements + i] = self._arr[i]
            self._head_ind = self._tail_ind + self._num_elements - 1
                
    def to_list(self):
        ind = self._tail_ind
        vals = []
        while True:
            vals.append(self._arr[ind])
            if ind == self._head_ind:
                break
            ind = self._advance_ind(ind)
        return vals[::-1]
        
    def enqueue(self, val):
        self._resize_if_necessary()
        if self._num_elements > 0:
            self._head_ind = self._advance_ind(self._head_ind)
        self._arr[self._head_ind] = val
        self._num_elements += 1
        return self
    
    def dequeue(self):
        val = self._arr[self._tail_ind]
        self._tail_ind = self._advance_ind(self._tail_ind)
        self._num_elements -= 1
        return val
    
    def get_num_elements(self):
        return self._num_elements

In [143]:
q = CircularQueue2(5)
print(q._arr)
q.enqueue(5)
print(q._arr)
q.enqueue(8)
print(q._arr)
q.enqueue(4)
q.enqueue(4)
q.enqueue(4)
q.enqueue(4)
print(q._arr)
q.dequeue()
print(q._arr)
q.enqueue(10)
print(q._arr)
q.enqueue(11)
print(q._arr)
print(q)

[None, None, None, None, None]
[5, None, None, None, None]
[5, 8, None, None, None]
res
[5, 8, 4, 4, 4, None, None, None, None, None]
([5, 8, 4, 4, 4, None, None, None, None, None], 0, 5)
[5, 8, 4, 4, 4, 4, None, None, None, None]
[5, 8, 4, 4, 4, 4, None, None, None, None]
[5, 8, 4, 4, 4, 4, 10, None, None, None]
[5, 8, 4, 4, 4, 4, 10, 11, None, None]
11->10->4->4->4->4->8


## 8.10 Implement a Queue with Max API 

Implement a queue with enqueue, dequeue, and max operations. The max operation returns the maximum element currently in the queue.

Hint: When can an element never be returned by max, regardless of future updates?

In [208]:
class Node:
    
    def __init__(self, val, next, prev=None):
        self.val = val
        self.next = next
        self.prev = prev

        
class Queue:
    
    def __init__(self):
        self._back = None
        self._front = None
    
    def __str__(self):
        return '->'.join([str(val) for val in self.to_list()])
        
    def to_list(self):
        vals = []
        node = self._back
        while node is not None:
            vals.append(node.val)
            node = node.next
        return vals
        
    def enqueue(self, val):
        self._back = Node(val, self._back)
        if self._front is None:
            self._front = self._back
        else:
            self._back.next.prev = self._back
        return self
    
    def dequeue(self):
        val = self._front.val
        self._front = self._front.prev
        if self._front is None:
            self._back = None
        else:
            self._front.next = None
        return val


class MaxQueue1:
    
    def __init__(self):
        self._queue = Queue()
        self._max = None
    
    def __str__(self):
        return str(self._queue)
        
    def to_list(self):
        return self._queue.to_list()
        
    def enqueue(self, val):
        self._max = max(self._max, val)
        self._queue.enqueue(val)
        return self
    
    def dequeue(self):
        val = self._queue.dequeue()
        if val == self._max:
            self._max = max(self.to_list())
        return val
    
    def max(self):
        return self._max
    
    
class MaxQueue2:
    
    def __init__(self):
        self._stack1 = MaxStack3()
        self._stack2 = MaxStack3()
    
    def __str__(self):
        return str('->'.join([str(val) for val in self.to_list()]))
        
    def to_list(self):
        return self._stack1.to_list() + self._stack2.to_list()
        
    def enqueue(self, val):
        self._stack1.push(val)
        return self
    
    def dequeue(self):
        if self._stack2.peek() is None:
            while self._stack1.peek() is not None:
                self._stack2.push(self._stack1.pop())
        return self._stack2.pop()
    
    def max(self):
        return max(self._stack1.max(), self._stack2.max())

In [209]:
q = MaxQueue2()
q.enqueue(5)
q.enqueue(8)
q.enqueue(7)
q.enqueue(6)
print(str(q), q.max())
q.dequeue()
print(str(q), q.max())
q.dequeue()
print(str(q), q.max())
q.dequeue()
print(str(q), q.max())

('5->11->7->6', 8)
('6->8->9', 8)
('6->8', 7)
('6', 6)


## 9.1 Check if a binary tree is height-balanced

A binary tree is said to be height-balanced if for each node in the tree, the difference in height of its left and right subtrees is at most one. A perfect binary tree is height-balanced, as is a complete binary tree. A height-balanced binary tree does not have to be perfect or complete -- see Figure 9.2 on the following page for an example.

Write a program that takes as input the root of a binary tree and checks whether the tree is height-balanced.

Hint: Think of a classic binary tree algorithm.

In [28]:
class BinaryTree:
    
    def __init__(self, key=None, val=None):
        self.key = key
        self.val = val
        self.parent = None
        self.left = None
        self.right = None
        
    def insert(self, key, val=None):
        if self.key is None:
            self.key = key
            self.val = val
            return self
        
        if key < self.key:
            if self.left is None:
                self.left = BinaryTree(key, val)
                self.left.parent = self
            else:
                self.left.insert(key, val)
        else:
            if self.right is None:
                self.right = BinaryTree(key, val)
                self.right.parent = self
            else:
                self.right.insert(key, val)
        
        return self
    
    def get_max_depth(self, prev_depth=0):
        if self.left is None:
            left_depth = prev_depth + 1
        else:
            left_depth = self.left.get_max_depth(prev_depth+1)
        
        if self.right is None:
            right_depth = prev_depth + 1
        else:
            right_depth = self.right.get_max_depth(prev_depth+1)
            
        depth = max(left_depth, right_depth)
        
        return depth
    
    def get_min_depth(self, prev_depth=0):
        if self.left is None or self.right is None:
            return prev_depth + 1
        else:
            return min(self.left.get_min_depth(prev_depth+1), self.right.get_min_depth(prev_depth+1))

        
def is_height_balanced(binary_tree):
    max_diff = binary_tree.get_max_depth() - binary_tree.get_min_depth()
    return max_diff <= 1

In [29]:
b = BinaryTree().insert(5).insert(6).insert(3)

In [30]:
is_height_balanced(b)

True

## 9.2 Test if a binary tree is symmetric

A binary tree is symmetric if you can draws a vertical line through the root and then the left subtree is the mirror image of the right subtree. The concept of a symmetric binary tree is illustrated in Figure 9.3.

Write a program that checks whether a binary tree is symmetric.

Hint: The definition of symmetry is recursive.

In [37]:
def is_symmetric(binary_tree):
    current_nodes = [binary_tree]
    
    while None not in current_nodes:
        for i in range(len(current_nodes) // 2):
            if current_nodes[i].key != current_nodes[len(current_nodes) - 1 - i].key:
                return False
        
        child_nodes = []
        for node in current_nodes:
            child_nodes.extend([node.left, node.right])
        
        current_nodes = child_nodes
    
    return True

In [39]:
b1 = BinaryTree().insert(5).insert(6).insert(3)
b2 = BinaryTree(5)
b2.left = BinaryTree(8)
b2.right = BinaryTree(8)
is_symmetric(b2)

True

## 9.4 Compute the LCA when nodes have parent pointers

Given two nodes in a binary tree, design an algorithm that computes their lowest common ancestor. Assume that each node has a parent pointer.

Hint: The problem is easy if both nodes are the same distance from the root.

## 9.11 Implement an inorder traversal with O(1) space

The direct implementation of an inorder traversal using recursion has O(h) space complexity, where h is the height of the three. Recursion can be removed with an explicit stack, but the space complexity remains O(h).

Write a nonrecursive program for computing the inorder traversal sequence for a binary tree. Assume nodes have parent fields.

Hint: How can you tell whether a node is a left child or right child of its parent?

## 9.12 Reconstruct a binary tree from traversal data

Many different binary trees yield the same sequence of keys in an inorder, preorder or postorder traversal. However, given an inorder traversal and one of any two other traversal orders of a binary tree, there exists a unique binary tree that yields those orders, assuming each node holds a distinct key. For example, the unique binary tree whose inorder traversal sequence is <F B A E H C D I G> and whose preorder traversal sequence is <H B F E A C D G I> is given in Figure 9.5 on the facing page.

Given an inorder traversal sequence and a preorder traversal sequence of a binary tree write a program to reconstruct the tree. Assume each node has a unique key.

Hint: Focus on the root.

## 9.16 Compute the right sibling tree

For this problem, assume that each binary tree node has an extra field, call it level-next, that holds a binary tree node (this field is distinct from the fields for the left and right children). The level-next field will be used to compute a map from nodes to their right siblings. The input is assumed to be a perfect binary tree. See Figure 9.6 for an example.

Write a program that takes a perfect binary tree, and sets each node's level-next field to the node on its right, if one exists.

Hint: Think of an appropriate traversal order.